In [1]:
import genjax
import jax
import jax.numpy as jnp

console = genjax.pretty()

In [11]:
def f(x):
    return x

In [12]:
genjax.inverse(f)(4.0)

Array(4., dtype=float32, weak_type=True)

In [13]:
genjax.ildj(f)(4.0)

Array(0., dtype=float32)

In [5]:
@genjax.trans_dist
def new_distribution(x):
    v = genjax.rv(Normal(x, 1.0))
    return jnp.exp(v)

In [6]:
key = jax.random.PRNGKey(314159)
key, w = new_distribution.estimate_logpdf(key, 3.0, 1.0)
w

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/mk/btkplz1n40q001dsy957srbh0000gn/T/ipykernel_51223/3230809007.py:2 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/mk/btkplz1n40q001dsy957srbh0000gn/T/ipykernel_51223/3230809007.py'                 │
│                                                                                                  │
│ /Users/mccoybecker/research/genjax/src/genjax/_src/generative_functions/distributions/coryx/lang │
│ .py:176 in estimate_logpdf                                                                       │
│                                                                                                  │
│   173 │   │   return key, (score, v)                                                             │
│   174 │                                                                                          │
│   175 │   def estimate_logpdf(self, key, v, *args, **kwargs):                                    │
│ ❱ 176 │   │   key, score = logpdf_transform(self.source)(key, v, args)                           │
│   177 │   │   return key, score                                                                  │
│   178                                                                                            │
│   179 ##############                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'logpdf_transform' is not defined